In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
### Distribution ✨ ❇ Demo 💪 ### 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### fancy logging variables, philosophically reminding us of the goal ###
fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
)

### pyfig ###
from pyfig import Pyfig

# arg = {
# 	'a_z':[4,], 
# 	'n_b': 256, 
# 	'n_sv': 16, 
# 	'n_pv': 16, 
# 	'n_corr': 20, 
# 	'n_step': 100000, 
# 	'log_metric_step': 1, 
# 	'exp_name':'demo',
# }

c = Pyfig(wb_mode='online', submit=False, run_sweep=False, notebook=True)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')


# from print import print
# print(c.d)

""" live plotting in another notebook """
""" copy lines and run in analysis while the exp is live """
# api = wandb.Api()
# run = api.run("<run-here>")
# c = run.config
# h = run.history()
# s = run.summary


# SOLVE THE CONUNDRUM 

init PlugIn classes
updating configuration
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: ff7b55a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
running script
setting exp_path
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: ff7b55a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xmax1 (hwat). Use `wandb login --relogin` to force relogin


🤖 1 GPUs available


' copy lines and run in analysis while the exp is live '

In [3]:
import torch
torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64
n_device = torch.cuda.device_count()
current_device = torch.cuda.current_device()
device = torch.cuda.device(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name(0)
print('cuda: ', device, 'n_dev: ', n_device, 'device', device, 'name: ', device_name)

cuda:  cuda n_dev:  1 device cuda name:  NVIDIA TITAN Xp


In [4]:
### model ###
from functools import partial
from hwat_func import Ansatz_fb
from torch import nn

import print

from hwat_func import init_r, get_center_points

dtype = torch.randn(1).dtype
c._convert(device, dtype=dtype)

n_e = c.data.n_e
center_points = get_center_points(c.data.n_e, c.app.a)
r = init_r(n_device, c.data.n_b, c.data.n_e, center_points, std=0.1).to(device)
dtype = r.dtype

print('r: ', r.shape, r.dtype)
r = r[0]  # single batch, single gpu, ❗ how to multi gpu

model_check = c.partial(Ansatz_fb, with_sign=True).to(device)
r_swap = r[0, [1,0]+[i for i in range(2, c.data.n_e)]]
lp0, s0 = model_check(r[0])
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())
r_swap = r[0, [i for i in range(0, c.data.n_e-2)]+[n_e-1,n_e-2]]
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())

print.print(c.d)

r:  torch.Size([1, 256, 4, 3]) torch.float64
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: ff7b55a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
anti-symmetry:  -17.823637621332686 -17.823637621332686 1.0 -1.0
anti-symmetry:  -17.823637621332686 -17.823637621332686 1.0 -1.0
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: ff7b55a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
{'TMP': PosixPath('dump/tmp'),
 'cap': 3,
 'commit_id': 'ff7b55a',
 'data': {'a': tensor([[0., 0., 0.]], device='cuda:0'),
          'a_z': tensor([4.], device='cuda:0'),
          'acc_target': 0.5,
          'charge': 0,
          'n_b': 256,
          'n_corr': 20,
          'n_d': 2,
          'n_e': 4,
          'n_equil': 10000,
          'n_u': 2,
          'spin': 0},
 'debug': False,
 'dtype': 'float32',
 'dump': PosixPath('dump'),
 'env': 'lumi',
 'exp_name': 'junk',
 'exp_path': PosixPath('dump/ex

In [5]:
### train step ###
from hwat_func import compute_ke_b, compute_pe_b
from functorch import vmap, make_functional, grad
import functorch

# def train_step(model, model_v, _params, r_step):

	

# 	v_tr = dict(
# 		params=params, 
#   		grads=grads,
# 		e=e, pe=pe, ke=ke,
# 		r=r_step
# 	)
# 	return v_tr


r = r.to(device)
center_points = center_points.to(device)

# v_tr = train_step(model, params, r)
# print.print({k:v.shape if isinstance(v, torch.Tensor) else len(v) for k,v in v_tr.items()})

### init variables ###
deltar = torch.tensor([0.02]).to(device)

print(f"""exp/actual | 
	cps    : {(c.data.n_e,3)}/{center_points.shape}
	r      : {(c.n_device, c.data.n_b, c.data.n_e, 3)}/{r.shape}
	deltar : {(1,)}/{deltar.shape}
""")

### init functions ### 
from hwat_func import sample_b

### train ###
import wandb
from hwat_func import keep_around_points
from utils import compute_metrix
from time import time
t0 = time()

### add in optimiser
model = c.partial(Ansatz_fb).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.001)
c.log_metric_step = 10
### fix sampler
### fix train step 
### metrix conversion
model_fn, params = make_functional(model)
model_v = vmap(model_fn, in_dims=(None, 0))

# print(c.wandb_c.)
wandb.define_metric("*", step_metric="tr/step")
print('Go see:', c._run.url)
v_tr = None
for step in range(1, c.n_step+1):
	r_keep = r.clone()
	with torch.no_grad():
     
     
		model_fn, params = make_functional(model)
		model_v = vmap(model_fn, in_dims=(None, 0))
		model_rv = lambda _r: model_v(params, _r).sum()

		r, acc, deltar = sample_b(model_v, params, r, deltar, n_corr=c.data.n_corr)  # ❗needs testing  ✅
		r = keep_around_points(r, center_points, l=2.) if step < 200 else r

		# with torch.no_grad():
		ke = compute_ke_b(model_rv, r)
		pe = compute_pe_b(r, c.app.a, c.app.a_z)
		# print(pe.shape, ke.shape)
		e = pe + ke
		e_mean_dist = torch.mean(torch.absolute(torch.median(e) - e))
		# print(e.shape, e_mean_dist.shape)
		e_clip = torch.clip(e, min=e-5*e_mean_dist, max=e+5*e_mean_dist)

	model.zero_grad()
	opt.zero_grad()
	loss = ((e_clip - e_clip.mean())*model_rv(r)).mean()
	loss.backward()
 
	opt.step()

	v_tr = dict(
		e=e, 
		pe=pe,
		ke=ke,
		r=r,
		params=params,
	)

	if not (step % c.log_metric_step):
		metrix = compute_metrix(v_tr)  # ❗ needs converting to torch, ie tree maps ✅
		wandb.log({'tr/step':step, **metrix})
	
	diff = time()-t0
	if diff > 10: 
		t0 = time()
		print(torch.absolute(r - r_keep).sum())
		print(r[0][:2])
		print.print({k:f'{v.mean().item():.4f}' for k,v in v_tr.items() if isinstance(v, torch.Tensor)} | {'step': step})
  

exp/actual | 
	cps    : (4, 3)/torch.Size([4, 3])
	r      : (1, 256, 4, 3)/torch.Size([256, 4, 3])
	deltar : (1,)/torch.Size([1])

Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: ff7b55a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
Go see: https://wandb.ai/hwat/hwat/runs/1au7d6p5
tensor(356.9424, device='cuda:0')
tensor([[-0.2679,  0.5838, -0.0088],
        [ 0.6103, -0.4952,  0.1720]], device='cuda:0')
{'e': '-10.8709', 'ke': '8.9920', 'pe': '-19.8629', 'r': '-0.0187', 'step': 11}
tensor(1275.8552, device='cuda:0')
tensor([[-0.8717,  0.5601, -0.0271],
        [ 0.6196,  1.2960, -1.5286]], device='cuda:0')
{'e': '-6.1854', 'ke': '6.9125', 'pe': '-13.0980', 'r': '-0.0187', 'step': 22}
tensor(1423.5954, device='cuda:0')
tensor([[-0.6559, -0.6175, -0.7900],
        [ 1.4435,  0.4194, -0.2540]], device='cuda:0')
{'e': '-8.5393', 'ke': '4.0690', 'pe': '-12.6083', 'r': '0.0021', 'step': 33}
tensor(1542.9798, device='cuda:0')
tensor([[-0.

KeyboardInterrupt: 

In [ ]:
# ```{toggle} env vars and jax debug config notes
# ❇️ Magic & debug not currently used

# %load_ext autoreload
# %autoreload 2
# %env CUDA_VISIBLE_DEVICES='3'
# %env "WANDB_NOTEBOOK_NAME" "run.ipynb" # ❕same as notebook

# from jax.config import config
# config.update('jax_disable_jit', True)
# ```